<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/hurr/ATL/2025/hurricane_forecast_subset_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install botocore
!pip install xarray
!pip install matplotlib

In [ ]:
import boto3
import botocore
from datetime import datetime, timedelta

import xarray as xr

import matplotlib.pyplot as plt

In [ ]:
def download_s3_file(BUCKET_NAME, KEY, fname):
    """
    Downloads a file from an AWS S3 bucket.

    Args:
        BUCKET_NAME (str): The name of the S3 bucket.
        KEY (str): The key (prefix) of the object in the bucket.
        fname (str): The filename of the object to download.

    Returns:
        str or None: The local filename if successful, None otherwise.
    """
    KEY_WITH_FNAME = f'{KEY}{fname}' # Construct the full key with the filename

    s3 = boto3.client('s3', config=botocore.config.Config(signature_version=botocore.UNSIGNED))

    try:
        s3.download_file(BUCKET_NAME, KEY_WITH_FNAME, fname) # Use the constructed key and fname for local filename
        print(f"Successfully downloaded {KEY_WITH_FNAME} to {fname}")
        return fname
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print(f"The object does not exist: {KEY_WITH_FNAME}")
        else:
            print(f"An error occurred: {e}")
        return None

# Inputs

In [ ]:
BUCKET_NAME = 'noaa-nws-rtofs-pds'
sysName = 'rtofs'

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/'

# forecast
#fPref, fSuff = ['rtofs_glo_3dz_f', '_6hrly_hvr_US_east.nc']
#hr_start, hr_end, freq = [6, 96, 6]
#save_data_path = save_data_path + 'RTOFS/forecast/'

#nowcast
fPref, fSuff = ['rtofs_glo_3dz_n', '_6hrly_hvr_US_east.nc']
hr_start, hr_end, freq = [6, 25, 6]
save_data_path = save_data_path + 'RTOFS/nowcast/'

date_start, date_end = ['20251101', '20251105']

# X, Y domain indices
x_start, x_end = [0, 600]
y_start, y_end = [0, 600]
z_start, z_end = [0, 1000]

# Jamaica? In x- and y- indices! The nc file is POORLY constructed.
x_pos = 310
y_pos_start, y_pos_end = [200, 250]

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

date_list = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

for date in date_list:
    dStr = date.strftime('%Y%m%d')
    KEY = f'{sysName}.{dStr}/'
    print(f'Working on {KEY}')
    filenames = [f'{fPref}{hour:03d}{fSuff}' for hour in range(hr_start, hr_end+1, freq)]
    for fname in filenames:
        print(f'Downloading {fname}')
        download_s3_file(BUCKET_NAME, KEY, fname)
        ds = xr.open_dataset(fname)
        ##ds_subset = ds.sel(X=slice(x_start, x_end), Y=slice(y_start, y_end))
        ds_subset = ds.sel(Depth=slice(z_start, z_end), Y=slice(y_start, y_end))
        fname_subset = fname.replace('.nc', '_subset.nc')
        ds_subset.to_netcdf(fname_subset)
        !rm {fname}

    subset_filenames = [fname.replace('.nc', '_subset.nc') for fname in filenames]
    ds = xr.open_mfdataset(subset_filenames)
    fName_save = f'{save_data_path}' + dStr + '.nc'
    print(f'Saving to {fName_save}')
    ds.to_netcdf(fName_save)

    for fname in subset_filenames:
      !rm {fname}

In [ ]:
#fig = plt.figure(figsize=(4, 2))
#ax = fig.add_subplot(111)

#im = ds.isel(MT=0).isel(Depth=0).sel(X=slice(150, 450), Y=slice(100, 350)).temperature.plot.contourf(ax=ax, x='X', y='Y', cmap='Spectral_r')

#ax.vlines(x=x_pos, ymin=y_pos_start, ymax=y_pos_end, color='k', linestyle='--', lw=2) # Add a vertical line segment

#ax.set_title(f'SST on {ds.MT[0].values}')

In [ ]:
#plot_grid = ds.isel(Depth=0).sel(X=slice(150, 450), Y=slice(100, 350)).temperature.plot.contourf(x='X', y='Y', col="MT", col_wrap=4, cmap='Spectral_r')

# Add lines to each subplot
#for ax in plot_grid.axs.flat:
#    ax.vlines(x=x_pos, ymin=y_pos_start, ymax=y_pos_end, color='k', linestyle='--') # Add a vertical line segment

# Access the colorbar axis and shrink its width
#plot_grid.cbar.ax.set_aspect(10) # Adjust the aspect ratio to make it thinner

In [ ]:
#from IPython.display import Image
#Image(url='https://www.nhc.noaa.gov/storm_graphics/AT13/refresh/AL132025_3day_earliest_reasonable_toa_no_wsp_34+png/251732_earliest_reasonable_toa_no_wsp_34.png')

In [ ]:
#plot_grid = ds.sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).temperature.plot.contourf(x='Y', y='Depth', yincrease=False, col="MT", col_wrap=4, levels=16, cmap='jet')

# Access the colorbar axis and shrink its width
#plot_grid.cbar.ax.set_aspect(10) # Adjust the aspect ratio to make it thinner

In [ ]:
#fig, ax = plt.subplots(figsize=(8, 4), nrows=1, ncols=2, sharey=True)

#ds.isel(MT=0).sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).temperature.plot.contourf(ax=ax[0], x='Y', y='Depth', yincrease=False, levels=16, cmap='jet')
#ax[0].set_title('20251025T00')

#ds.isel(MT=15).sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).temperature.plot.contourf(ax=ax[1], x='Y', y='Depth', yincrease=False, levels=16, cmap='jet')
#ax[1].set_title('2025-10-29T00')

In [ ]:
#fig, ax = plt.subplots(figsize=(8, 4), nrows=1, ncols=2, sharey=True)

#ds.isel(MT=0).sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).salinity.plot.contourf(ax=ax[0], x='Y', y='Depth', yincrease=False, levels=16, cmap='jet')
#ax[0].set_title('20251025T00')

#ds.isel(MT=15).sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).salinity.plot.contourf(ax=ax[1], x='Y', y='Depth', yincrease=False, levels=16, cmap='jet')
#ax[1].set_title('2025-10-29T00')

In [ ]:
#plot_grid = ds.sel(X=x_pos, method='nearest').sel(Depth=slice(0, 300), Y=slice(y_pos_start, y_pos_end)).salinity.plot.contourf(x='Y', y='Depth', yincrease=False, col="MT", col_wrap=4, levels=11, cmap='jet')

# Access the colorbar axis and shrink its width
#plot_grid.cbar.ax.set_aspect(10) # Adjust the aspect ratio to make it thinner

# Create an animation

In [ ]:
#!apt-get update && apt-get install -y imagemagick

In [ ]:
#import os
#import matplotlib.pyplot as plt

# Create a directory to save the frames
#if not os.path.exists('temp_frames'):
#    os.makedirs('temp_frames')

#for i, time_step in enumerate(ds.MT):
#    # Create a new figure and axes for each frame
#    fig, ax = plt.subplots()

#    # Select the data for the current time step and spatial slice
#    temp_data = ds.isel(Depth=0).sel(X=slice(150, 450), Y=slice(100, 350)).temperature.sel(MT=time_step)

#    # Plot the data on the axes
#    cf = ax.contourf(temp_data.X, temp_data.Y, temp_data.values, vmin=25, vmax=32, cmap='Spectral_r')

#    # Add a colorbar to the current frame
#    fig.colorbar(cf, ax=ax)

#    # Add a title to each frame
#    ax.set_title(f'Temperature at {time_step.values}')

#    # Save the current frame as a PNG file
#    frame_filename = f'temp_frames/frame_{i:03d}.png'
#    fig.savefig(frame_filename)

#    # Close the figure to free up memory
#    plt.close(fig)

In [ ]:
#!convert -delay 30 -loop 0 temp_frames/frame_*.png temperature_animation.gif

# Use xmovie

In [ ]:
#!pip install xmovie
#!apt-get update && apt-get install -y ffmpeg netcdf4

In [ ]:
#!pip install netcdf4

In [ ]:
#from xmovie import Movie

In [ ]:
#movie = Movie(ds.isel(Depth=0).sel(X=slice(150, 450), Y=slice(100, 350)).salinity,
#              framedim='MT',
#              input_check=False,
#              x='X',
#              y='Y',
#              vmin=34,
#              vmax=38,
#              cmap='jet')

In [ ]:
#movie.save('salinity_animation_xmovie.gif')

In [ ]:
#movie = Movie(ds.isel(Depth=0).sel(X=slice(150, 450), Y=slice(100, 350)).temperature,
#              framedim='MT',
#              input_check=False,
#              x='X',
#              y='Y',
#              vmin=25,
#              vmax=32,
#              cmap='Spectral_r')

In [ ]:
#movie.save('temperature_animation_xmovie.gif')